In [1]:
!pip install transformers
!pip install peft


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load the base model and tokenizer
base_model_id = "unsloth/Qwen2-1.5B-Instruct"
fine_tuned_model_id = "Muhammad7865253/qwen-1.5B-medical-QA"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_id)

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_id)

# Load the fine-tuned LoRA model
model = PeftModel.from_pretrained(base_model, fine_tuned_model_id)

# Check if a GPU is available and move the model to the GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Function to generate responses
def generate_response(instruction, query):
    prompt = f"Instruction: {instruction}\nUser Query: {query}\nAI Response:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=1.5,
        top_p=0.9,
        do_sample=True,
        no_repeat_ngram_size=3  # Prevent n-gram repetition
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt from the response to avoid repetition
    response = response.split('AI Response:')[-1].strip()

    return response

# Example usage
instruction = "Provide helpful, empathetic mental health advice."
queries = [
    "How can I build emotional resilience to handle future challenges better?",
    "How can I stop overthinking when I’m stressed out?",
    "How can I better cope with feelings of loneliness?"
]

for query in queries:
    print(f"User: {query}")
    response = generate_response(instruction, query)
    print(f"AI: {response}\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

User: How can I build emotional resilience to handle future challenges better?
AI: Hi, Welcome to HCM! First of all I need to understand why do you need the help. Is your current life challenging? Do you experience low mood, anxiety and anger at times? If any of above things apply then first consult psychiatrist for evaluation and treatment.I will suggest some relaxation techniques also which are proven for many patients like - Deep breathing technique / meditation / yoga or pranayama / massage (if comfortable with it).These may be helpful as follow - Take deep deep breaths while in sitting, lie supine position, close eyes and breathe slowly for several 2-5 minutes daily. You can take one glass water on hands after that do the exercise for few days if feeling more relaxed.Mindfulness is also a useful thing to practice which will increase self-esteem.Hope i have answered your question.Wish speedy recovery. Take Care.Regards.Dr Manoj K Yadav, Psychiatrist.(Counseling). For further querie

In [3]:
def generate_response(instruction, query):
    prompt = f"Instruction: {instruction}\nUser Query: {query}\nAI Response:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=1.5,
        top_p=0.9,
        do_sample=True,
        no_repeat_ngram_size=3  # Prevent n-gram repetition
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt from the response to avoid repetition
    response = response.split('AI Response:')[-1].strip()

    # Format the response into a more structured output
    organized_response = "Here’s a structured response:\n\n"
    response_lines = response.split('. ')  # Split sentences
    for idx, line in enumerate(response_lines, start=1):
        if line.strip():  # Ignore empty lines
            organized_response += f"{idx}. {line.strip()}.\n"

    return organized_response

# Example usage
instruction = "Provide helpful, empathetic mental health advice."
queries = [
    "How can I build emotional resilience to handle future challenges better?",
    "How can I stop overthinking when I’m stressed out?",
    "How can I better cope with feelings of loneliness?"
]

for query in queries:
    print(f"User: {query}")
    response = generate_response(instruction, query)
    print(f"AI:\n{response}\n")


User: How can I build emotional resilience to handle future challenges better?
AI:
Here’s a structured response:

1. Hi, thanks for asking.
2. Building of the Resiliency needs practice and continuous self improvement through all your actions.
3. It has to do with having control in our thinking processes, over thoughts we have in times of pressure.
4. In this regard, you must practice relaxation methods like meditation.
5. Also exercise regularly as it is good way for releasing energy from muscles.
6. Do not be too harsh or rigid on oneself; you should treat each incident as a teaching moment.
7. This process can only develop slowly but surely, by consistent action.Hope this helps.Regards,Dr Nupur Roshan, Psychologist and Family Counselor, Institute of Mental Health & Behavioural Sciences, New Delhi, India.
8. 201-845-1666.
9. You need not worry at all about what is happening now and you are doing nothing wrong at all and there is every chance that the situation could turn around for yo